In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 11GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # vector input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # label input
    in_label = Input(shape=(1,))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((image_shape[0], image_shape[1], image_shape[2]))(li)
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image, li])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(latent_dim)(li)
    li = Reshape([latent_dim])(li)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen, li])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = Bidirectional(LSTM(200, activation='relu', return_sequences=True))(b)
    # output
    out_layer = Dense(3)(d)
    # define model input & output
    model = Model([in_image, in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat, in_label])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out, in_label])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat, in_label], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
	return [X1, X2, X3-1]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	img, vec, label = dataset
	# choose random instances
	ix = randint(0, img.shape[0], n_samples)
	# retrieve selected images
	X1, X2, X3 = img[ix], vec[ix], label[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2, X3], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, label, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input, label])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
def generate_drvact_text(drvact):
    text = '[warnings] drvact label is not defined ...'
    if (drvact == 1):
        text = 'Go'
    elif (drvact == 2):
        text = 'Turn Left'
    elif (drvact == 3):
        text = 'Turn Right'
    elif (drvact == 4):
        text = 'U-turn'
    elif (drvact == 5):
        text = 'Left LC'
    elif (drvact == 6):
        text = 'Right LC'
    elif (drvact == 7):
        text = 'Avoidance'
    return text

In [13]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_real_img, X_real_vec, X_label], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fake_vec, _ = generate_fake_samples(g_model, X_real_img, X_label, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_real_img = (X_real_img + 1) / 2.0
    pyplot.figure(figsize=(32.0, 20.0))
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_real_img[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_fake_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_real_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [14]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=3):
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(dataset[0]) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_real_img, X_real_vec, X_label], y_real = generate_real_samples(dataset, n_batch)
        # generate a batch of fake samples
        X_fake_vec, y_fake = generate_fake_samples(g_model, X_real_img, X_label, n_batch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_real_img, X_real_vec, X_label], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_real_img, X_fake_vec, X_label], y_fake)
        X_lat = generate_latent_points(latent_dim, n_batch)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_real_img, X_lat, X_label], [y_real, X_real_vec])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(bat_per_epo) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')

In [15]:
%%time
# load image data
dataset = load_real_samples('./dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape, dataset[2].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3) (16805,)
CPU times: user 22 s, sys: 1.1 s, total: 23.1 s
Wall time: 23.1 s


In [16]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
# %%
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)
# %%
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [17]:
#see model summary
print(g_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 128, 128, 64) 3136        input_4[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_7 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 64, 64, 128)  131200      leaky_re_lu_7[0][0]              
____________________________________________________________________________________________

In [ ]:
# train model
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.156] d2[3.559] g[319.382]
>2, d1[0.009] d2[1.356] g[393.189]
>3, d1[0.027] d2[1.094] g[226.162]
>4, d1[0.111] d2[0.775] g[294.085]
>5, d1[0.015] d2[0.562] g[622.086]
>6, d1[0.032] d2[0.934] g[378.602]
>7, d1[0.231] d2[1.269] g[91.048]
>8, d1[0.068] d2[0.179] g[505.067]
>9, d1[0.054] d2[0.224] g[285.847]
>10, d1[0.071] d2[0.348] g[303.754]
>11, d1[0.093] d2[0.475] g[387.046]
>12, d1[0.001] d2[0.216] g[405.523]
>13, d1[0.233] d2[0.156] g[210.688]
>14, d1[0.000] d2[0.466] g[470.146]
>15, d1[0.173] d2[0.707] g[256.931]
>16, d1[0.209] d2[1.074] g[252.240]
>17, d1[0.129] d2[1.531] g[502.183]
>18, d1[1.014] d2[0.309] g[115.477]
>19, d1[0.351] d2[0.643] g[267.713]
>20, d1[0.324] d2[0.390] g[195.275]
>21, d1[0.123] d2[0.359] g[452.876]
>22, d1[0.167] d2[0.675] g[209.532]
>23, d1[0.067] d2[0.157] g[207.835]
>24, d1[0.053] d2[0.230] g[226.776]
>25, d1[0.097] d2[1.750] g[234.898]
>26, d1[0.273] d2[0.131] g[261.831]
>27, d1[0.014] d2[0.157] g[418.810]
>28, d1[0.086] d2[0.262] g[274.883]
>2

>226, d1[0.525] d2[0.258] g[202.685]
>227, d1[0.496] d2[0.348] g[145.812]
>228, d1[0.395] d2[0.404] g[118.986]
>229, d1[0.304] d2[0.367] g[230.363]
>230, d1[0.461] d2[0.178] g[288.016]
>231, d1[0.373] d2[0.233] g[120.215]
>232, d1[0.287] d2[0.232] g[207.400]
>233, d1[0.753] d2[0.264] g[223.959]
>234, d1[0.365] d2[0.289] g[180.585]
>235, d1[0.294] d2[0.350] g[186.534]
>236, d1[0.486] d2[0.236] g[128.742]
>237, d1[0.292] d2[0.350] g[147.739]
>238, d1[0.445] d2[0.268] g[359.718]
>239, d1[0.400] d2[0.405] g[415.157]
>240, d1[0.496] d2[0.240] g[113.557]
>241, d1[0.294] d2[0.696] g[130.017]
>242, d1[0.370] d2[0.227] g[275.228]
>243, d1[0.804] d2[0.253] g[182.444]
>244, d1[0.379] d2[0.259] g[260.912]
>245, d1[0.330] d2[0.261] g[186.513]
>246, d1[0.733] d2[0.739] g[192.642]
>247, d1[0.582] d2[0.311] g[86.206]
>248, d1[0.529] d2[0.458] g[81.554]
>249, d1[0.434] d2[0.245] g[290.529]
>250, d1[0.326] d2[0.223] g[207.972]
>251, d1[0.330] d2[0.499] g[50.464]
>252, d1[0.339] d2[0.299] g[321.058]
>253

>448, d1[0.377] d2[0.374] g[105.434]
>449, d1[0.299] d2[0.333] g[384.884]
>450, d1[0.413] d2[0.361] g[75.139]
>451, d1[0.430] d2[0.305] g[115.048]
>452, d1[0.238] d2[0.299] g[72.761]
>453, d1[0.393] d2[0.515] g[216.327]
>454, d1[0.289] d2[0.391] g[110.312]
>455, d1[0.356] d2[0.449] g[159.656]
>456, d1[0.251] d2[0.405] g[122.307]
>457, d1[0.352] d2[0.430] g[222.885]
>458, d1[0.327] d2[0.372] g[180.661]
>459, d1[0.336] d2[0.305] g[49.890]
>460, d1[0.319] d2[0.384] g[171.710]
>461, d1[0.313] d2[0.462] g[253.051]
>462, d1[0.304] d2[0.372] g[188.906]
>463, d1[0.342] d2[0.435] g[124.487]
>464, d1[0.389] d2[0.313] g[189.075]
>465, d1[0.257] d2[0.475] g[150.247]
>466, d1[0.229] d2[0.356] g[225.189]
>467, d1[0.270] d2[0.376] g[214.858]
>468, d1[0.371] d2[0.325] g[51.927]
>469, d1[0.349] d2[0.372] g[178.448]
>470, d1[0.261] d2[0.462] g[154.152]
>471, d1[0.291] d2[0.366] g[125.428]
>472, d1[0.403] d2[0.414] g[278.805]
>473, d1[0.357] d2[0.346] g[156.945]
>474, d1[0.322] d2[0.445] g[255.021]
>475,

>672, d1[0.311] d2[0.480] g[137.599]
>673, d1[0.353] d2[0.404] g[173.701]
>674, d1[0.364] d2[0.342] g[127.039]
>675, d1[0.377] d2[0.445] g[83.016]
>676, d1[0.347] d2[0.304] g[156.476]
>677, d1[0.374] d2[0.270] g[100.932]
>678, d1[0.484] d2[0.318] g[192.930]
>679, d1[0.367] d2[0.334] g[71.598]
>680, d1[0.361] d2[0.468] g[165.045]
>681, d1[0.339] d2[0.440] g[158.405]
>682, d1[0.264] d2[0.409] g[166.689]
>683, d1[0.338] d2[0.298] g[188.227]
>684, d1[0.270] d2[0.346] g[118.556]
>685, d1[0.311] d2[0.278] g[115.992]
>686, d1[0.288] d2[0.342] g[175.165]
>687, d1[0.387] d2[0.443] g[154.372]
>688, d1[0.297] d2[0.313] g[142.289]
>689, d1[0.366] d2[0.265] g[98.859]
>690, d1[0.325] d2[0.299] g[69.123]
>691, d1[0.337] d2[0.420] g[138.067]
>692, d1[0.337] d2[0.361] g[138.120]
>693, d1[0.328] d2[0.331] g[84.323]
>694, d1[0.276] d2[0.310] g[111.688]
>695, d1[0.406] d2[0.302] g[94.872]
>696, d1[0.342] d2[0.350] g[205.536]
>697, d1[0.321] d2[0.476] g[167.807]
>698, d1[0.252] d2[0.345] g[132.087]
>699, d

>895, d1[0.296] d2[0.487] g[192.714]
>896, d1[0.287] d2[0.391] g[112.988]
>897, d1[0.386] d2[0.385] g[52.912]
>898, d1[0.322] d2[0.244] g[86.048]
>899, d1[0.301] d2[0.444] g[130.363]
>900, d1[0.352] d2[0.393] g[156.288]
>901, d1[0.294] d2[0.359] g[126.081]
>902, d1[0.295] d2[0.369] g[206.811]
>903, d1[0.385] d2[0.415] g[138.304]
>904, d1[0.359] d2[0.334] g[100.886]
>905, d1[0.332] d2[0.405] g[174.691]
>906, d1[0.302] d2[0.318] g[145.285]
>907, d1[0.301] d2[0.423] g[231.516]
>908, d1[0.418] d2[0.303] g[171.439]
>909, d1[0.320] d2[0.379] g[146.065]
>910, d1[0.325] d2[0.369] g[235.364]
>911, d1[0.404] d2[0.430] g[104.863]
>912, d1[0.310] d2[0.314] g[139.610]
>913, d1[0.316] d2[0.390] g[91.400]
>914, d1[0.320] d2[0.308] g[94.504]
>915, d1[0.324] d2[0.329] g[125.890]
>916, d1[0.323] d2[0.336] g[65.431]
>917, d1[0.296] d2[0.412] g[185.968]
>918, d1[0.361] d2[0.379] g[159.793]
>919, d1[0.336] d2[0.337] g[181.071]
>920, d1[0.321] d2[0.411] g[137.900]
>921, d1[0.285] d2[0.334] g[121.672]
>922, 

...

>41699, d1[0.356] d2[0.347] g[20.528]
>41700, d1[0.351] d2[0.356] g[31.135]
>41701, d1[0.340] d2[0.332] g[13.034]
>41702, d1[0.337] d2[0.369] g[24.049]
>41703, d1[0.338] d2[0.322] g[16.825]
>41704, d1[0.347] d2[0.343] g[17.695]
>41705, d1[0.344] d2[0.348] g[14.378]
>41706, d1[0.365] d2[0.353] g[41.419]
>41707, d1[0.328] d2[0.351] g[12.432]
>41708, d1[0.361] d2[0.366] g[19.380]
>41709, d1[0.330] d2[0.356] g[24.534]
>41710, d1[0.360] d2[0.342] g[21.485]
>41711, d1[0.338] d2[0.357] g[17.842]
>41712, d1[0.351] d2[0.328] g[29.212]
>41713, d1[0.328] d2[0.338] g[8.595]
>41714, d1[0.356] d2[0.370] g[20.179]
>41715, d1[0.338] d2[0.334] g[18.657]
>41716, d1[0.356] d2[0.360] g[25.066]
>41717, d1[0.348] d2[0.346] g[30.758]
>41718, d1[0.352] d2[0.360] g[29.207]
>41719, d1[0.345] d2[0.355] g[26.037]
>41720, d1[0.338] d2[0.343] g[19.138]
>41721, d1[0.366] d2[0.343] g[23.435]
>41722, d1[0.344] d2[0.342] g[27.659]
>41723, d1[0.360] d2[0.353] g[48.894]
>41724, d1[0.367] d2[0.332] g[30.687]
>41725, d1[0.

>41915, d1[0.343] d2[0.365] g[24.986]
>41916, d1[0.348] d2[0.350] g[23.154]
>41917, d1[0.338] d2[0.356] g[21.521]
>41918, d1[0.352] d2[0.368] g[32.729]
>41919, d1[0.331] d2[0.365] g[10.259]
>41920, d1[0.347] d2[0.350] g[22.412]
>41921, d1[0.352] d2[0.334] g[12.434]
>41922, d1[0.354] d2[0.362] g[24.622]
>41923, d1[0.338] d2[0.356] g[28.732]
>41924, d1[0.343] d2[0.355] g[19.429]
>41925, d1[0.338] d2[0.349] g[16.960]
>41926, d1[0.336] d2[0.331] g[15.286]
>41927, d1[0.331] d2[0.348] g[9.739]
>41928, d1[0.342] d2[0.360] g[12.790]
>41929, d1[0.344] d2[0.349] g[10.626]
>41930, d1[0.329] d2[0.347] g[9.690]
>41931, d1[0.373] d2[0.340] g[21.196]
>41932, d1[0.363] d2[0.356] g[11.811]
>41933, d1[0.338] d2[0.339] g[33.043]
>41934, d1[0.338] d2[0.347] g[33.243]
>41935, d1[0.344] d2[0.356] g[16.395]
>41936, d1[0.370] d2[0.340] g[17.199]
>41937, d1[0.346] d2[0.340] g[22.344]
>41938, d1[0.360] d2[0.346] g[28.350]
>41939, d1[0.358] d2[0.351] g[10.006]
>41940, d1[0.342] d2[0.355] g[14.738]
>41941, d1[0.3

>42132, d1[0.372] d2[0.349] g[12.793]
>42133, d1[0.330] d2[0.339] g[13.159]
>42134, d1[0.331] d2[0.356] g[24.417]
>42135, d1[0.345] d2[0.330] g[11.172]
>42136, d1[0.340] d2[0.364] g[30.075]
>42137, d1[0.352] d2[0.348] g[23.567]
>42138, d1[0.338] d2[0.332] g[15.585]
>42139, d1[0.341] d2[0.367] g[19.684]
>42140, d1[0.337] d2[0.336] g[21.081]
>42141, d1[0.331] d2[0.356] g[13.453]
>42142, d1[0.327] d2[0.333] g[21.982]
>42143, d1[0.342] d2[0.362] g[14.789]
>42144, d1[0.356] d2[0.358] g[19.568]
>42145, d1[0.334] d2[0.343] g[31.275]
>42146, d1[0.333] d2[0.342] g[26.950]
>42147, d1[0.351] d2[0.345] g[28.669]
>42148, d1[0.356] d2[0.347] g[12.246]
>42149, d1[0.340] d2[0.339] g[14.786]
>42150, d1[0.355] d2[0.350] g[15.387]
>42151, d1[0.331] d2[0.362] g[13.011]
>42152, d1[0.353] d2[0.334] g[19.958]
>42153, d1[0.333] d2[0.347] g[20.744]
>42154, d1[0.345] d2[0.360] g[8.005]
>42155, d1[0.364] d2[0.361] g[37.796]
>42156, d1[0.366] d2[0.342] g[16.754]
>42157, d1[0.332] d2[0.337] g[12.386]
>42158, d1[0.

>42348, d1[0.365] d2[0.341] g[9.058]
>42349, d1[0.332] d2[0.353] g[24.199]
>42350, d1[0.384] d2[0.363] g[13.561]
>42351, d1[0.357] d2[0.358] g[16.628]
>42352, d1[0.352] d2[0.359] g[17.473]
>42353, d1[0.360] d2[0.338] g[22.713]
>42354, d1[0.360] d2[0.365] g[13.640]
>42355, d1[0.342] d2[0.349] g[14.537]
>42356, d1[0.327] d2[0.352] g[16.321]
>42357, d1[0.344] d2[0.347] g[19.309]
>42358, d1[0.341] d2[0.341] g[19.997]
>42359, d1[0.357] d2[0.341] g[23.621]
>42360, d1[0.361] d2[0.375] g[23.257]
>42361, d1[0.359] d2[0.356] g[16.996]
>42362, d1[0.331] d2[0.334] g[17.067]
>42363, d1[0.347] d2[0.354] g[22.439]
>42364, d1[0.344] d2[0.343] g[13.280]
>42365, d1[0.341] d2[0.332] g[13.955]
>42366, d1[0.355] d2[0.331] g[39.129]
>42367, d1[0.353] d2[0.346] g[15.949]
>42368, d1[0.365] d2[0.344] g[24.218]
>42369, d1[0.345] d2[0.344] g[22.938]
>42370, d1[0.327] d2[0.368] g[26.842]
>42371, d1[0.334] d2[0.363] g[19.088]
>42372, d1[0.340] d2[0.349] g[17.268]
>42373, d1[0.353] d2[0.340] g[15.234]
>42374, d1[0.

>42566, d1[0.345] d2[0.358] g[17.264]
>42567, d1[0.331] d2[0.366] g[25.539]
>42568, d1[0.333] d2[0.335] g[26.363]
>42569, d1[0.362] d2[0.378] g[36.985]
>42570, d1[0.331] d2[0.341] g[29.272]
>42571, d1[0.349] d2[0.328] g[23.154]
>42572, d1[0.328] d2[0.308] g[12.831]
>42573, d1[0.375] d2[0.360] g[20.351]
>42574, d1[0.333] d2[0.342] g[17.536]
>42575, d1[0.349] d2[0.348] g[37.779]
>42576, d1[0.367] d2[0.377] g[28.333]
>42577, d1[0.335] d2[0.341] g[6.492]
>42578, d1[0.365] d2[0.361] g[12.915]
>42579, d1[0.333] d2[0.343] g[34.926]
>42580, d1[0.365] d2[0.323] g[19.942]
>42581, d1[0.331] d2[0.339] g[57.487]
>42582, d1[0.337] d2[0.363] g[12.767]
>42583, d1[0.343] d2[0.355] g[25.802]
>42584, d1[0.334] d2[0.343] g[17.528]
>42585, d1[0.345] d2[0.330] g[23.769]
>42586, d1[0.342] d2[0.351] g[20.244]
>42587, d1[0.333] d2[0.343] g[18.059]
>42588, d1[0.351] d2[0.344] g[18.630]
>42589, d1[0.353] d2[0.362] g[22.399]
>42590, d1[0.343] d2[0.345] g[11.428]
>42591, d1[0.365] d2[0.349] g[9.645]
>42592, d1[0.3

In [ ]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()